In [1]:
##Hoping to remove reliance on ArcGIS for ground cover processing

# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy as np
import tempfile
from osgeo import gdal, gdalconst
from osgeo import ogr
import glob
import sys
sys.path.append('../Modules')
import zonalMeanByBincount
import os
import datetime
import calendar
#import scipy
#from scipy import ndimage
#import matplotlib.pyplot as plt

#requiredStartDate = datetime.date(1990,1,1)
#requiredEndDate = datetime.date(1992,12,31)

#Path to shapefile to use for clip/mask
#pathToShapefile = 'D:/FractionalCover/MackayWhit_Buffer_10km_Albers.shp'
#pathToShapefile = 'D:/FractionalCover/MaryBurnett_Buffer_10km_Albers.shp'

pathToZonesRaster = 'D:/BurnettMary/spatial_RC8/BM_CreatedSubcats.asc'
pathToZonesCategories = 'D:/BurnettMary/spatial_RC8/BM_CreatedSubcats_categories.csv'

tempRasterPath = os.path.join(tempfile.gettempdir(), "resampledData.tif")

#Do one raster only for now
#rasterPathName = 'D:/FractionalCover/GroundCover/lztmre_qld_m198909198911_dixa2.tif'
#Do one raster only for now
#patchPathName = 'D:/FractionalCover/Patches/lztmre_rreef_m198909198911_dj4a2.tif'

#fractionalCoverDir = r'\\nrm02002\EXCHANGE\Fractional Ground Cover\Ground Cover'
#patchDir = r'\\nrm02002\EXCHANGE\Fractional Ground Cover\Ground Cover Patch'
fractionalCoverDir = r'D:\FractionalCover\GroundCover'
patchDir = r'D:\FractionalCover\Patches'

outputDir = r'D:\BurnettMary\Covers'
#outputCSV = r'D:\BurnettMary\Covers\FullCoverMeans.csv'
outputCSV = r'D:\BurnettMary\Covers\TestCoverMeans.csv'

requiredStartDate = datetime.date(1989,9,1)
requiredEndDate = datetime.date(1990,5,31)
#requiredStartDate = datetime.date(1987,9,1)
#requiredEndDate = datetime.date(2017,2,28)

#Use these to control the saved files
saveCover = False
#saveCFact = True

extensions = ('*.tif', '*.tiff') # the tuple of file types

availCoverRasters = []
availPatchRasters = []

for theExt in extensions:
    ###
    availCoverRasters.extend(glob.glob(fractionalCoverDir + "/" + theExt))
    availPatchRasters.extend(glob.glob(patchDir + "/" + theExt))

#availCoverRasters = glob.glob(fractionalCoverDir + "/*.tif")
#availPatchRasters = glob.glob(patchDir)

#Final NoData Val: can't use 0, as that is a 'valid' cover val....
ourNoData = 240

#some data storage bits and pieces
allStats = {}
catchIDs = {}
foundStartDates = []


In [2]:
#load the zones raster
zoneRas = gdal.Open(pathToZonesRaster)
zone_GeoTrans = zoneRas.GetGeoTransform()
zonesRasterArray = zoneRas.ReadAsArray()
ZoneNoDataVal = zoneRas.GetRasterBand(1).GetNoDataValue()
uniqZones = np.unique(zonesRasterArray)
### Going to give up on masking zones, as the bincount unique stuff will include it the NoData Val labels anyway


#MaskedZones = np.ma.masked_where(zonesRasterArray==ZoneNoDataVal, zonesRasterArray)
#MaskedZones = zonesRasterArray
#uniqZones = np.unique(MaskedZones)

rasterIdxToNames = pandas.read_csv(pathToZonesCategories)
## Add the row for the NoDataVal
rasterIdxToNames.loc[-1] = [ZoneNoDataVal, "NoVal"]
rasterIdxToNames.index = rasterIdxToNames.index + 1  # shifting index
rasterIdxToNames = rasterIdxToNames.sort_index()

#uniqZones.remove(ZoneNoDataVal)

##Place holders
#for zID in uniqZones:
#    allStats[zID] = {}

#zone_GeoTrans

#Open shapefile
#clipPolys = ogr.Open(pathToShapefile)
#lyr = clipPolys.GetLayer(0)
#clipExt = lyr.GetExtent()

#Print feature count
#lyr.GetFeatureCount()

In [3]:
#index = np.argwhere(uniqZones==ZoneNoDataVal)
#uniqZones = np.delete(uniqZones, index)
#uniqZones = uniqZones.sorted()
#uniqZones = np.unique(MaskedZones)

#Place holders
#for zID in uniqZones:
#    allStats[zID] = {}


In [4]:
print(uniqZones)

[-9999     1     2     3     4     5     6     7     8     9    10    11
    12    13    14    15    16    17    18    19    20    21    22    23
    24    25    26    27    28    29    30    31    32    33    34    35
    36    37    38    39    40    41    42    43    44    45    46    47
    48    49    50    51    52    53    54    55    56    57    58    59
    60    61    62    63    64    65    66    67    68    69    70    71
    72    73    74    75    76    77    78    79    80    81    82    83
    84    85    86    87    88    89    90    91    92    93    94    95
    96    97    98    99   100   101   102   103   104   105   106   107
   108   109   110   111   112   113   114   115   116   117   118   119
   120   121   122   123   124   125   126   127   128   129   130   131
   132   133   134   135   136   137   138   139   140   141   142   143
   144   145   146   147   148   149   150   151   152   153   154   155
   156   157   158   159   160   161   162   163   

In [5]:
#print(str(len(uniqZones)))
#MaskedZones = np.ma.masked_where(zonesRasterArray==ZoneNoDataVal, zonesRasterArray)
#index = np.argwhere(uniqZones==ZoneNoDataVal)
#print(index)
#uniqZones = np.delete(uniqZones, index)
#print(str(len(uniqZones)))

In [6]:
def reProjectAndReSampleRaster(templateRaster, dataRaster):
    dataProj = dataRaster.GetProjection()
    dataRef = dataRaster.GetRasterBand(1)
    referenceProj = templateRaster.GetProjection()
    referenceTrans = templateRaster.GetGeoTransform()
    x = templateRaster.RasterXSize
    y = templateRaster.RasterYSize
    #Keep data type simple for storage space, we know we're dealing with integers here
    #Allowing nearest neighbour resampling
    output = gdal.GetDriverByName("GTiff").Create(tempRasterPath, x, y, 1, dataRef.DataType)
    ###Need a 'No Data Val' that is different to Zero, because that will cause dramas for zero-cover...
    #Can't use -9999!, use 240 as it is higher than any valid data value
    output.GetRasterBand(1).SetNoDataValue(ourNoData)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)
    #gdalconst.GRA_Bilinear
    gdal.ReprojectImage(dataRaster,output,dataProj,referenceProj,gdalconst.GRA_NearestNeighbour)
    

In [7]:
def processCoverFile(fullFileName):
    
    fName = os.path.basename(fullFileName)
    
    beginningIdx = fName.index('_m') + 2
    endingIdx = beginningIdx + 12
    #print("Found at: " + str(beginningIdx))
    #print("Found at: " + fName[beginningIdx:endingIdx])
    
    searchBit = fName[beginningIdx:endingIdx]
    
    theStartDate = datetime.date(int(fName[beginningIdx:beginningIdx+4]),int(fName[beginningIdx+4:beginningIdx+6]),1)
    
    endYear = fName[beginningIdx+6:beginningIdx+10]
    endMonth = fName[beginningIdx+10:beginningIdx+12]
    
    maxDay = calendar.monthrange(int(endYear), int(endMonth))[1]
    
    theEndDate = datetime.date(int(endYear),int(endMonth),maxDay)
    
    if (((theStartDate >= requiredStartDate) and (theStartDate <= requiredEndDate)) or ((theEndDate >= requiredStartDate) and (theEndDate <= requiredEndDate))):
        #Yep, process this fellow
        now = datetime.datetime.now()
        print(now.strftime("%Y-%m-%d %H:%M")+ " Processing cover for period: " + str(theStartDate) + " to " + str(theEndDate))
        
        alreadyHaveDate = False
        for existSDate in foundStartDates:
            if existSDate == str(theStartDate):
                alreadyHaveDate = True
                break
            
        if alreadyHaveDate == False:
            foundStartDates.append(str(theStartDate))
        
        #open cover raster
        print("Opening fractional cover: " + fName)
        covRas = gdal.Open(fullFileName)
        covRas_GeoTrans = covRas.GetGeoTransform()
        
        if (covRas_GeoTrans != zone_GeoTrans):
            #Need to reproject/resample
            print("Cover raster resolution info didn't match template, attempted a resample")
            reProjectAndReSampleRaster(zoneRas, covRas)
            #Load resampled raster from temp area
            covRas = gdal.Open(tempRasterPath)
            covRas_GeoTrans = covRas.GetGeoTransform()
            #Do we check????
            if (covRas_GeoTrans == zone_GeoTrans):
                print("Cover raster resolution now matches")
            
        
        #find & open patch raster
        patchFileName = None
        for patchFile in availPatchRasters:
            if(os.path.basename(patchFile).find(searchBit) != -1):
                patchFileName = patchFile
                break
        
        
        print("Opening patch data: " + os.path.basename(patchFileName))
        patchRas = gdal.Open(patchFileName)
        patchRas_GeoTrans = patchRas.GetGeoTransform()
        
        if (patchRas_GeoTrans != zone_GeoTrans):
            print("Patch raster resolution info didn't match template, attempted a resample")
            reProjectAndReSampleRaster(zoneRas, patchRas)
            #Load resampled raster from temp area
            patchRas = gdal.Open(tempRasterPath)
            patchRas_GeoTrans = patchRas.GetGeoTransform()
            if (patchRas_GeoTrans == zone_GeoTrans):
                print("Patch raster resolution now matches")
        
        
        #Should be no need to do clipping now, re-sampling would've done it?
        
        #newRasterInfo = shapeExtent_To_Raster_Box(clipExt, origRas_GeoTrans)
        #patchRasterInfo = shapeExtent_To_Raster_Box(clipExt, origPatchRas.GetGeoTransform())
        
        print("Working...")
        sourceRastArray = covRas.GetRasterBand(1).ReadAsArray()
        patchRastArray = patchRas.GetRasterBand(1).ReadAsArray()

        ##clip = rast[ulY:lrY, ulX:lrX]
        #clipSourceRast = sourceRast[newRasterInfo[8]:newRasterInfo[9],newRasterInfo[6]:newRasterInfo[7]]
        #clipPatchRast = patchRast[patchRasterInfo[8]:patchRasterInfo[9],patchRasterInfo[6]:patchRasterInfo[7]]


        ##Introduce mask of NoDatas
        ##This will include ocean as well as gaps
        ##clipSourceRast = numpy.ma.masked_values(clipSourceRast, origRas.GetRasterBand(1).GetNoDataValue())
        ##clipPatchRast = numpy.ma.masked_values(clipPatchRast, origPatchRas.GetRasterBand(1).GetNoDataValue())
        #clipSourceRast = numpy.ma.masked_where((clipSourceRast < 100) | (clipSourceRast > 200) | (clipSourceRast ==  origRas.GetRasterBand(1).GetNoDataValue()), clipSourceRast)
        #clipPatchRast = numpy.ma.masked_where((clipPatchRast < 100) | (clipPatchRast > 200) | (clipPatchRast == origPatchRas.GetRasterBand(1).GetNoDataValue()), clipPatchRast)
        clipSourceRast = np.ma.masked_where((sourceRastArray < 100) | (sourceRastArray > 200) | (sourceRastArray ==  covRas.GetRasterBand(1).GetNoDataValue()), sourceRastArray)
        clipPatchRast = np.ma.masked_where((patchRastArray < 100) | (patchRastArray > 200) | (patchRastArray == patchRas.GetRasterBand(1).GetNoDataValue()), patchRastArray)

        
        #Free up some memory
        sourceRastArray = None
        patchRastArray = None
        
        clipPatchRast = np.ma.masked_values(clipPatchRast, patchRas.GetRasterBand(1).GetNoDataValue())
        
        insertedPatch = np.ma.masked_values(clipSourceRast.filled(clipPatchRast), patchRas.GetRasterBand(1).GetNoDataValue())
        
        #Need to change data type of array so that we can go above values of 255
        patchedMask = np.array(insertedPatch, dtype='f')
        patchedMask = np.ma.masked_values(patchedMask, patchRas.GetRasterBand(1).GetNoDataValue())
        
        ###Need a 'No Data Val' that is different to Zero, because that will cause dramas for zero-cover...
        #Can't use -9999!, go 240
        patchedMask = patchedMask.filled(ourNoData)
        patchedMask = np.ma.masked_values(patchedMask, ourNoData)
        
        #Important! Use 200.0, not '200', to ensure we retain 'float' data type
        gCov = 200.0-patchedMask
        
        visPatched = 0.00925 * np.power(gCov, 2)

        ##This essential to allow us insert NoDatas in final product... again....
        visPatched = visPatched.filled(ourNoData)
        visPatched = np.ma.masked_values(visPatched, ourNoData)
        
        #img_masked = np.where(visPatched, visPatched, np.nan)
        
        ####Now we can do zonal stats......
        ##Just use numpy arrays
        print("Doing zonal stats on " + str(len(uniqZones)) + " unique zones")
        #outStats = scipy.ndimage.measurements.mean(visPatched, labels=MaskedZones, index=uniqZones)
        #outStats = scipy.ndimage.measurements.mean(visPatched.filled(np.nan), labels=MaskedZones, index=uniqZones)
        
        #This uses the Bincount method (detailed in comments below), but in a module we can use from other places
        outStats = zonalMeanByBincount.zonalMeanByBincount(zonesRasterArray, visPatched)
        
        
        
#         #Try our own bincount, without Module
#         unique_labels, new_labels = np.unique(zonesRasterArray, return_inverse=True)

#         #print("Labs:")
#         #print(unique_labels)
        
#         ##new_labels = new_labels.filled(ourNoData)
#         #new_labels = np.ma.masked_values(new_labels, ourNoData)
#         ##vpRav = visPatched.ravel()
#         ##vpRav.filled(ourNoData)
#         ##new_labels = np.ma.masked_where(vpRav != ourNoData, new_labels)
#         #counts = np.bincount(new_labels)
#         newPatch = visPatched.ravel()
#         newPatch = newPatch.filled(np.nan)
#         keep = ~np.isnan(newPatch)
#         #newPatch = np.ma.masked_values(newPatch, 0.0)
        
#         #print("Is masked???: " + str(np.ma.is_masked(newPatch)))
        
#         counts = np.bincount(new_labels[keep])
        
#         #sums = np.bincount(new_labels, weights=visPatched.ravel())
#         sums = np.bincount(new_labels[keep], weights=newPatch[keep])
        
#         outStats = sums/np.asanyarray(counts).astype(np.float)
        
        
        ###This method similar to what zonal stats from web examples does, but its slow
        #for zID in uniqZones:
        #    if zID == ZoneNoDataVal:
        #        print("Ignoring value: " + str(zID))
        #        continue
                
            
        #    masked = np.ma.masked_where(np.not_equal(zonesRasterArray, zID), visPatched)
        #    allStats[zID][str(theStartDate)] = float(masked.mean())
        #    print("Added to catchID:" + str(zID) + " meanCover: " + str(allStats[zID][str(theStartDate)]))
        
        
        ###This looping option even slower than repeated masking
        #outStatsCount = {}
        #outStatsSum = {}
        
        #for unZID in uniqZones:
        #    outStatsCount[unZID] = 0
        #    outStatsSum[unZID] = 0
        
        #i = 0
        #while i < zoneRas.RasterYSize:
            
        #    j = 0
            
        #    while j < zoneRas.RasterXSize:
                
        #        #if MaskedZones[i,j] == 'masked':
        #        if zonesRasterArray[i,i] == ZoneNoDataVal:
        #            continue
                
        #        if visPatched[i,j] == ourNoData:
        #            continue
                
                
        #        outStatsCount[MaskedZones[i,j]] += 1
        #        outStatsSum[MaskedZones[i,j]] += visPatched[i,j]
        
        
        #outStats = []
        
        #for unZID in uniqZones:
        #    if outStatsCount[unZID] == 0:
        #        outStats.append('NA')
        #    else:
        #        outStats.append(outStatsSum[unZID]/outStatsCount[unZID])
            

        allStats[str(theStartDate)] = outStats

        #print(outStats)


        #Cfactor....
        #cFactor = numpy.exp(-0.799 - (0.0474 * visPatched) + (0.000449 * numpy.power(visPatched, 2)) - (0.0000052 * numpy.power(visPatched, 3)))
        #cFactor = numpy.ma.masked_values(cFactor, ourNoData)

        if (saveCover):
            ###
            outputFile = os.path.join(outputDir, "cov_" + searchBit + ".tif")
            
            print("Saving: " + outputFile)

            #Integer will be fine for cover bands
            #GDT_Byte is 8 bit unsigned (apparently)
            
            #outDS = gdal.GetDriverByName("GTiff").Create(outputFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Byte)
            #outDS.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))
            
            
            
            outDS = gdal.GetDriverByName("GTiff").Create(outputFile, zoneRas.RasterXSize, zoneRas.RasterYSize, 1, gdal.GDT_Byte)
            outDS.SetGeoTransform((zone_GeoTrans[0], zone_GeoTrans[1], zone_GeoTrans[2], zone_GeoTrans[3], zone_GeoTrans[4], zone_GeoTrans[5]))
            
            #outDS.SetProjection(origRas.GetProjection())
            outBand = outDS.GetRasterBand(1)

            # write the data
            outBand.WriteArray(visPatched)

            # flush data to disk, set the NoData value and calculate stats
            outBand.FlushCache()
            outBand.SetNoDataValue(ourNoData)
            outDS.SetProjection(zoneRas.GetProjection())
            
            del outDS, outBand

        #if (saveCFact):
        #    ##
        #    outputCFactFile = os.path.join(outputDir, "cf_" + searchBit + ".tif")
        #    
        #    print("Saving: " + outputCFactFile)
        #    
        #    outDSCFact = gdal.GetDriverByName("GTiff").Create(outputCFactFile, newRasterInfo[4], newRasterInfo[5], 1, gdal.GDT_Float32)
        #    outDSCFact.SetGeoTransform((newRasterInfo[0], newRasterInfo[2], 0, newRasterInfo[1], 0, newRasterInfo[3]))
        #
        #    outDSCFact.SetProjection(origRas.GetProjection())
        #    outBandCFact = outDSCFact.GetRasterBand(1)
        #
        #    outBandCFact.WriteArray(cFactor)
        #    outBandCFact.FlushCache()
        #    outBandCFact.SetNoDataValue(ourNoData)
        #
        #    # georeference the image and set the projection (ROB: don't do GeoTransform, we've already done that)
        #    ##outDs.SetGeoTransform(inDs.GetGeoTransform())
        #    outDSCFact.SetProjection(origRas.GetProjection())

        #    #Without this 'del', the output file doesn't get populated with values???
        #    del outDSCFact, outBandCFact

            
            
        clipPatchRast = None
        clipSourceRast = None
        patchedMask = None
        gCov = None
        visPatched = None
        cFactor = None
        

In [8]:

for theFile in availCoverRasters:
    processCoverFile(theFile)

print("All done")


2018-09-04 12:38 Processing cover for period: 1989-09-01 to 1989-11-30
Opening fractional cover: lztmre_qld_m198909198911_dixa2.tif
Cover raster resolution info didn't match template, attempted a resample
Cover raster resolution now matches
Opening patch data: lztmre_rreef_m198909198911_dj4a2.tif
Patch raster resolution info didn't match template, attempted a resample
Patch raster resolution now matches
Working...
Doing zonal stats on 683 unique zones
2018-09-04 12:38 Processing cover for period: 1989-12-01 to 1990-02-28
Opening fractional cover: lztmre_qld_m198912199002_dixa2.tif
Cover raster resolution info didn't match template, attempted a resample
Cover raster resolution now matches
Opening patch data: lztmre_rreef_m198912199002_dj4a2.tif
Patch raster resolution info didn't match template, attempted a resample
Patch raster resolution now matches
Working...
Doing zonal stats on 683 unique zones
2018-09-04 12:39 Processing cover for period: 1990-03-01 to 1990-05-31
Opening fractiona

In [9]:
#Now to get at the data...
print(foundStartDates)
#rasterIdxToNames

['1989-09-01', '1989-12-01', '1990-03-01']


In [10]:
#dFrame = pandas.DataFrame()
#dFrame['zones'] = uniqZones

#rasterIdxToNames.sort_values(by=['Ras_ID'])


#dFrame

#sTd = foundStartDates.sort()


for eachSDate in foundStartDates:
    print(str(eachSDate) + " Length: " + str(len(allStats[eachSDate])))
    
    if len(allStats[eachSDate]) == 683:
        rasterIdxToNames[eachSDate] = allStats[eachSDate]
    


#for unZID in uniqZones:
#    for eachSDate in sTd:
#        dFrame[str(unZID)] = allStats[zID][eachSDate]

#for eachSDate in foundStartDates.sorted():
#    for unZID in uniqZones:
#        dFrame[str(unZID)] = allStats[zID][eachSDate]

1989-09-01 Length: 683
1989-12-01 Length: 683
1990-03-01 Length: 683


In [11]:
#rasterIdxToNames


In [12]:
rasterIdxToNames.to_csv(outputCSV, index=False)
now = datetime.datetime.now()
print("File exported at: " + now.strftime("%Y-%m-%d %H:%M"))

File exported at: 2018-09-04 12:39
